In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
#Adapted from: 
# lqr_neural_error_dynamics.ipynb

import scipy.io as sio
import scipy.stats as sio_stat
import scipy.interpolate
import numpy as np
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt 

import os
import pickle
import sys
import copy

import time
import pylab as pl
from IPython import display

from bmi_dynamics_code import behavior_co_obs as bmi_b
from bmi_dynamics_code import data_for_v as preeyacode
from bmi_dynamics_code import util as bmi_util

import timeit
from sklearn import decomposition as skl_decomp

import statsmodels.api as sm
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge

%matplotlib notebook

In [4]:
%matplotlib notebook

In [5]:
load_dir = r'D:\Dropbox\BMI_co_obs_paper\data'
load_path = os.path.join(load_dir, 'dynamics_for_v.pkl')

with open(load_path, 'rb') as f: 
    r = pickle.load(f)

In [6]:
r.keys()

[('grom', 8, 'b'),
 ('jeev', 2, 'b'),
 ('jeev', 1, 'b'),
 ('grom', 4, 'b'),
 ('grom', 5, 'b'),
 ('grom', 2, 'b'),
 ('grom', 5, 'A'),
 ('grom', 2, 'A'),
 ('grom', 1, 'A'),
 ('jeev', 3, 'A'),
 ('grom', 1, 'b'),
 ('grom', 7, 'b'),
 ('jeev', 0, 'b'),
 ('grom', 0, 'b'),
 ('grom', 0, 'A'),
 ('jeev', 0, 'A'),
 ('grom', 6, 'b'),
 ('grom', 8, 'A'),
 ('jeev', 3, 'b'),
 ('grom', 7, 'A'),
 ('grom', 3, 'A'),
 ('jeev', 1, 'A'),
 ('grom', 6, 'A'),
 ('grom', 3, 'b'),
 ('jeev', 2, 'A'),
 ('grom', 4, 'A')]

In [ ]:
load_dir = r'D:\Dropbox\BMI_co_obs_paper\data'
load_path = os.path.join(load_dir, 'decoders_for_v.pkl')
# os.path.exists(load_path)

with open(load_path, 'rb') as f: 
    r = pickle.load(f)
# for (k,v) in r.items():
#     exec(k+'=v')

In [ ]:
r.keys()

In [ ]:
#Code used to save the decoders: 

In [ ]:
# def save_decoders(): 
#     decoder_dict = {}
#     for i_a, animal in enumerate(['grom', 'jeev']):
#         for day_ix in range(analysis_config.data_params['%s_ndays'%animal]):
#             ### get decoder: 
#             if animal == 'grom':
#                 F, KG = util_fcns.get_grom_decoder(day_ix)
#             elif animal == 'jeev':
#                 F = util_fcns.get_jeev_F(day_ix)
#                 ### Returns a 2 x Nneurons estimated Kalman Gain matrix 
#                 KG_sub = util_fcns.get_jeev_decoder(day_ix)
#                 ### Populate the 7 x Nneurons matrix
#                 ### Confirmed from previous analysis that neural push only 
#                 ### ever effects velocity (on current timestep), not position 
#                 KG = np.zeros((7, KG_sub.shape[1]))
#                 KG[[3, 5], :] = KG_sub.copy()
#             #### Format of matrices is [Xpos,Zpos,Ypos, Xvel,Zvel,Yvel, offset]: 7 dim
#             #### This is a 2D task, so Zpos/Zvel are irrelevant, so removing them:
#             keep_ix = [0, 2, 3, 5, 6] 
#             KG_xy = KG[keep_ix, :]
#             F_xy = F[np.ix_(keep_ix, keep_ix)]
#             ### Shoudl yield a 5 x Nneurons and 5x5 matrix: 
#             assert(KG_xy.shape[0] == F_xy.shape[0] == F_xy.shape[1] == 5)
#             decoder_dict[animal, day_ix, 'KG'] = KG_xy.copy()
#             decoder_dict[animal, day_ix, 'F'] = F_xy.copy()
#     pickle.dump(decoder_dict, open(os.path.join(analysis_config.config['BMI_DYN'], 'scripts', 'decoders_for_v.pkl'), 'wb'))

In [ ]:
# def save_decoders(): 
#     decoder_dict = {}
#     dynamics_dict = {}
#     for i_a, animal in enumerate(['grom', 'jeev']):
#         model_fname = analysis_config.config[animal+'_pref']+'tuning_models_'+animal+'_model_set'+str(6)+'_.pkl'
#         model_data = pickle.load(open(model_fname, 'rb'))
#         for day_ix in range(analysis_config.data_params['%s_ndays'%animal]):
#             ### get decoder: 
#             if animal == 'grom':
#                 F, KG = util_fcns.get_grom_decoder(day_ix)
#             elif animal == 'jeev':
#                 F = util_fcns.get_jeev_F(day_ix)
#                 ### Returns a 2 x Nneurons estimated Kalman Gain matrix 
#                 KG_sub = util_fcns.get_jeev_decoder(day_ix)
#                 ### Populate the 7 x Nneurons matrix
#                 ### Confirmed from previous analysis that neural push only 
#                 ### ever effects velocity (on current timestep), not position 
#                 KG = np.zeros((7, KG_sub.shape[1]))
#                 KG[[3, 5], :] = KG_sub.copy()
#             #### Format of matrices is [Xpos,Zpos,Ypos, Xvel,Zvel,Yvel, offset]: 7 dim
#             #### This is a 2D task, so Zpos/Zvel are irrelevant, so removing them:
#             keep_ix = [0, 2, 3, 5, 6] 
#             KG_xy = KG[keep_ix, :]
#             F_xy = F[np.ix_(keep_ix, keep_ix)]
#             ### Shoudl yield a 5 x Nneurons and 5x5 matrix: 
#             assert(KG_xy.shape[0] == F_xy.shape[0] == F_xy.shape[1] == 5)
#             decoder_dict[animal, day_ix, 'KG'] = KG_xy.copy()
#             decoder_dict[animal, day_ix, 'F'] = F_xy.copy()
#             ### Get dynamics 
#             fold = 0
#             mod = model_data[day_ix, 'hist_1pos_0psh_2spksm_1_spksp_0', fold, 0, 'model']
#             dynamics_dict[animal, day_ix, 'A'] =  mod.coef_.copy()
#             dynamics_dict[animal, day_ix, 'b'] =  mod.intercept_.copy()
#     pickle.dump(decoder_dict, open(os.path.join(analysis_config.config['BMI_DYN'], 'scripts', 'decoders_for_v.pkl'), 'wb'))
#     pickle.dump(dynamics_dict, open(os.path.join(analysis_config.config['BMI_DYN'], 'scripts', 'dynamics_for_v.pkl'), 'wb'))

In [ ]:
r.keys()